In [2]:
import pandas as pd

C:\Users\ensem\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ensem\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\ensem\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [1]:
#ファイル読み込みからデータフレーム成型まで一連の処理を関数にする。
def make_df(path):
    '''
    見積書を読込み、データフレームを成型する。
    <input>
        path : 読込みたいエクセルファイルのパス(str)
    <output>
        df_summary : 必要事項のみを抽出したデータフレーム(DataFrame)
    '''
    data = pd.read_excel(path, engine="openpyxl")
    #品名はindexでいうと12,　開始列は0列目, 終了列は4列目であることを利用してスライス
    data1 = data.iloc[12:, :5]
    data2 = data1.reset_index(drop=True)

    #0列目が初めてNaNになるインデックスを取得する
    nan_idx = data2[data2.iloc[:, 0].isnull()==True].index.tolist()
    
    #nan_idxの１つめより下の行をスライスで除去
    data3 = data2.iloc[:nan_idx[0], :]

    #カラムに入れたい0番目の値をリストに格納し、カラムとして定義する。
    cols = data3.iloc[0, :].tolist()
    data3.columns = cols

    #0番目の行は不要なのでスライスで除去
    data4 = data3.iloc[1:, :]

    #購入日の取得
    buy_date = data.iloc[0, 6]

    #仕入先の取得
    supplier = data.iloc[2, 0]

    #データフレームへの追加
    data4['購入日'] = buy_date
    data4['仕入先'] = supplier

    #並べ替えの方法はいくつかあるが、ここでは購入日より右、左でデータフレームを分割し、再度結合する方法をとる。
    data4_r = data4.iloc[:, 5:]
    data4_l = data4.iloc[:, :5]

    #データフレームの結合
    data5 = pd.concat([data4_r, data4_l], axis=1)

    return data5

In [4]:
file_paht = 'example/mitumori_ex1.xlsx'
df = make_df(file_paht)
df

<ipython-input-1-fb376b083566>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['購入日'] = buy_date
<ipython-input-1-fb376b083566>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['仕入先'] = supplier


,購入日,仕入先,品 名,単 価,数 量,値引き額,金 額
1,2022-02-20,株式会社〇×▽,トランジスタ,50,2,0,100
2,2022-02-20,株式会社〇×▽,IC,100,10,50,950
3,2022-02-20,株式会社〇×▽,トランス,2000,2,100,3900
